## P10

In [10]:
from collections import namedtuple

Point = namedtuple('Point', ['x', 'y']) 

belt = set()
with open('p10_input.txt') as f:
    for y, line in enumerate(f):
        for x, c in enumerate(line):
            if c == '#':
                belt.add(Point(x, y))

In [12]:
Point(0, 2) in belt

False

In [59]:
def los(p1, p2):
    l = set()
    sx, sy = p1
    lx, ly = p2
    if abs(lx - sx) == 0:
        my = min(sy, ly)
        ly = max(sy, ly)
        for dy in range(ly - my + 1):
            l.add(Point(sx, my + dy))
    else:
        m = (ly - sy) / abs(lx - sx)
        for dx in range(abs(lx - sx) + 1):
            nx = sx+dx if lx > sx else sx - dx
            l.add(Point(nx, sy + m*dx))
    return l
    
cnts = []
for src in belt:
    cnt = 0
    for dst in belt - {src}:
        if not los(src, dst) & belt - {src, dst}:
            cnt += 1
    cnts.append(cnt)

print(max(cnts))

347


In [54]:

los(Point(0, 1), Point(4, 5)) & belt - {Point(0, 1)}

set()

## P9

In [ ]:
with open('p9_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
prog1 = [109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99]
prog2 = [1102,34915192,34915192,7,4,7,99,0]
prog3 = [104,1125899906842624,99]

p1 = Prog(prog1)
p2 = Prog(prog2)
p3 = Prog(prog3)

p = Prog(prog)
p.input_deque.append(1)

def run(p):
    while not p.finished:
        p.run()
    print(p.output_deque)
    
run(p)

In [ ]:
p = Prog(prog)
p.input_deque.append(2)
run(p)

In [ ]:
from collections import defaultdict, deque

def opc(s):
    s_ = '{:05d}'.format(int(s))
    code = int(s_[-2:])
    params = s_[0:3]
    return code, params[::-1]

class Prog:
    def __init__(self, prog):
        self.prog = prog.copy()
        self.base_prog = prog.copy()
        self.offset = 0
        self.input_deque = deque()
        self.output_deque = deque()
        self.finished = False
        self.memory = defaultdict(lambda: 0)
        self.rel_base = 0
        
    def run(self):
        _continue = True
#         print('Run with ', self.input_deque)
        while _continue:
            _continue, _offset = self.op2(self.prog)
            self.offset += _offset
            
    def mem_read(self, s, addr, relative=False):
        if addr < len(s):
            return s[addr]
        else:
            return self.memory[addr]
    
    def mem_write(self, s, addr_, val, relative=False):
        if relative:
            addr = self.rel_base + addr_
        else:
            addr = addr_
        if addr < len(s):
            s[addr] = val
        else:
            self.memory[addr] = val
        
    def op2(self, s):
        offset = self.offset
        code, params = opc(s[offset])
#         print(s[offset], code, offset, params)
        _off = 4
        if code == 3 or code == 4 or code == 9:
            _off = 2
        if code == 5 or code == 6:
            _off = 3

        if code == 99:
            self.finished = True
            return False, 1
        pr = s[offset:offset+_off].copy()

        ppr = []
        for i, val in enumerate(pr[1:]):
    #         print(i, val)
            if int(params[i]) == 0:
                addr = int(val)
                ppr.append(int(self.mem_read(s, addr)))
            elif int(params[i]) == 1:
                ppr.append(int(val))
            else:
                addr = self.rel_base + int(val)
                ppr.append(int(self.mem_read(s, addr)))

    #     print(ppr)

        if code == 3:
    #         in_ = input()
            in_ = self.input_deque.popleft()
            in_ = int(in_)
            self.mem_write(s, int(pr[-1]), in_, int(params[0])==2)
            return True, _off
        elif code == 4:
#             print('>>>', s[int(pr[-1])])
            self.output_deque.append(ppr[0])
            return False, _off
        elif code == 5:
            flag = ppr[0]
            if flag > 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 6:
            flag = ppr[0]
            if flag == 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 9:
            self.rel_base += ppr[0]
            return True, _off

        if code == 1:
            self.mem_write(s,int(pr[-1]), ppr[0] + ppr[1], int(params[-1])==2)
            return True, _off
        elif code == 2:
            self.mem_write(s,int(pr[-1]), ppr[0] * ppr[1], int(params[-1])==2)
            return True, _off
        elif code == 7:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] < ppr[1] else 0, int(params[-1])==2)
            return True, _off
        elif code == 8:
            self.mem_write(s,int(pr[-1]), 1 if ppr[0] == ppr[1] else 0, int(params[-1])==2)
            return True, _off
        elif code == 99:
            return False, 1
        else:
            print('Error')
            return False

## P8

In [ ]:
with open('p8_input.txt') as f:
    img_buffer = f.read().strip()

In [ ]:
w = 25
h = 6
nl = int(len(img_buffer) / (w*h))
print('Image is {}x{}x{}'.format(w, h, nl))

In [ ]:
img = []
ptr = 0
for _ in range(nl):
    layer = []
    for _ in range(h):
        for _ in range(w):
            layer.append(img_buffer[ptr])
            ptr += 1
    img.append(layer)


In [ ]:
from collections import Counter

counters = []
for l in range(nl):
    counters.append(Counter(img[l]))
    
target = min(cnt['0'] for cnt in counters)

for l in range(nl):
    if counters[l]['0'] == target:
        print(counters[l]['1']*counters[l]['2'])

In [ ]:
acc = img[0].copy()

In [ ]:
for l in range(1, nl):
    for ptr in range(len(acc)):
        if int(acc[ptr]) == 2:
            if int(img[l][ptr]) < 2:
                acc[ptr] = img[l][ptr]

In [ ]:
def print_img(acc):
    ptr = 0
    for hp in range(h):
        for wp in range(w):
            print(acc[ptr], end='')
            ptr += 1
        print()
        
print_img(acc)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np_img = np.zeros((h,w,3), np.uint8)

ptr = 0
for hp in range(h):
    for wp in range(w):
        if int(acc[ptr]) == 1:
            np_img[hp, wp] = (255, 255, 255)
        ptr += 1

matplotlib.pyplot.imshow(np_img)

## P7

In [ ]:
prog1 = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
prog2 = [3,23,3,24,1002,24,10,24,1002,23,-1,23,
101,5,23,23,1,24,23,23,4,23,99,0,0]

In [ ]:
from collections import deque
inp = deque([3, 0])
inp.popleft()

In [ ]:
run_prog(prog1)

In [ ]:
with open('p7_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
params = [0, 1, 2, 3, 4]
from itertools import permutations

perms = permutations(params)

output = []

for perm in perms:
    inp = deque([0])
    for p in perm:
        inp.appendleft(p)
        prog_ = prog.copy()
        run_prog(prog_, inp)
    output.append(inp[0])
print(max(output))

In [ ]:
class Prog:
    def __init__(self, prog):
        self.prog = prog.copy()
        self.base_prog = prog.copy()
        self.offset = 0
        self.input_deque = deque()
        self.output_deque = deque()
        self.finished = False
        
    def run(self):
        _continue = True
#         print('Run with ', self.input_deque)
        while _continue:
            _continue, _offset = self.op2(self.prog)
            self.offset += _offset
        
    def op2(self, s):
        offset = self.offset
        code, params = opc(s[offset])
    #     print(s[offset], code, offset, params)
        _off = 4
        if code == 3 or code == 4:
            _off = 2
        if code == 5 or code == 6:
            _off = 3

        if code == 99:
            self.finished = True
            return False, 1
        pr = s[offset:offset+_off].copy()

        ppr = []
        for i, val in enumerate(pr[1:]):
    #         print(i, val)
            if int(params[i]) == 0:
                ppr.append(int(s[int(val)]))
            else:
                ppr.append(int(val))

    #     print(ppr)

        if code == 3:
    #         in_ = input()
            in_ = self.input_deque.popleft()
            in_ = int(in_)
            s[int(pr[-1])] = in_
            return True, _off
        elif code == 4:
#             print('>>>', s[int(pr[-1])])
            self.output_deque.append(s[int(pr[-1])])
            return False, _off
        elif code == 5:
            flag = ppr[0]
            if flag > 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off
        elif code == 6:
            flag = ppr[0]
            if flag == 0:
                new_addr = ppr[1]
                return True, new_addr - offset
            else:
                return True, _off

        if code == 1:
            s[int(pr[-1])] = ppr[0] + ppr[1]
            return True, _off
        elif code == 2:
            s[int(pr[-1])] = ppr[0] * ppr[1]
            return True, _off
        elif code == 7:
            s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
            return True, _off
        elif code == 8:
            s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
            return True, _off
        elif code == 99:
            return False, 1
        else:
            print('Error')
            return False

In [ ]:
# prog3 = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
# 27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]

# prog4 = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
# -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
# 53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]


# phase_params = [9, 8, 7, 6, 5]
# phase_params = [9, 7, 8, 5, 6]

def run_feedback(phase_params):
    amps = [Prog(prog) for i in range(5)]

    for i in range(5):
        amps[i].input_deque.append(phase_params[i])

    amps[0].input_deque.append(0)

    finished = False
    res = 0
    while not finished:
        for i in range(5):
            amps[i].run()
            next_idx = (i+1) % 5
#             print(next_idx)
            if amps[i].output_deque:
                res = amps[i].output_deque.pop()
                amps[next_idx].input_deque.append(res)
            finished = amps[i].finished
#             if finished:
#                 print('Prog ', i, ' finished')
    return res
    
params = [5, 6, 7, 8, 9]
from itertools import permutations

perms = permutations(params)

output = []

for perm in perms:
    output.append(run_feedback(perm))
print(max(output))

In [ ]:
12 % 10

In [ ]:
def op2(s, offset, inp):
    code, params = opc(s[offset])
#     print(s[offset], code, offset, params)
    _off = 4
    if code == 3 or code == 4:
        _off = 2
    if code == 5 or code == 6:
        _off = 3
    
    if code == 99:
        return False, -1
    pr = s[offset:offset+_off].copy()
    
    ppr = []
    for i, val in enumerate(pr[1:]):
#         print(i, val)
        if int(params[i]) == 0:
            ppr.append(int(s[int(val)]))
        else:
            ppr.append(int(val))
            
#     print(ppr)
    
    if code == 3:
#         in_ = input()
        in_ = inp.popleft()
        in_ = int(in_)
        s[int(pr[-1])] = in_
        return True, _off
    elif code == 4:
        print('>>>', s[int(pr[-1])])
        inp.append(s[int(pr[-1])])
        if phase_params:
            p = phase_params.popleft()
            inp.appendleft(p)
#             phase_params.append(p)
#         phase_params.append(p)
#         print(phase_params, inp)
        return False, _off
    elif code == 5:
        flag = ppr[0]
        if flag > 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    elif code == 6:
        flag = ppr[0]
        if flag == 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    
    if code == 1:
        s[int(pr[-1])] = ppr[0] + ppr[1]
        return True, _off
    elif code == 2:
        s[int(pr[-1])] = ppr[0] * ppr[1]
        return True, _off
    elif code == 7:
        s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
        return True, _off
    elif code == 8:
        s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
        return True, _off
    elif code == 99:
        return False, 1
    else:
        print('Error')
        return False

def run_prog2(prog, inp):
    offset = 0
    _continue = True
    while _continue:
        _continue, _offset = op2(prog, offset, inp)
        #print('Operation at {}'.format(offset))
        offset += _offset
    if _offset == -1:
        return True
    else:
        return False

In [ ]:
def op(s, offset, inp):
    code, params = opc(s[offset])
#     print(s[offset], code, offset, params)
    _off = 4
    if code == 3 or code == 4:
        _off = 2
    if code == 5 or code == 6:
        _off = 3
    
    if code == 99:
        return False, 1
    pr = s[offset:offset+_off].copy()
    
    ppr = []
    for i, val in enumerate(pr[1:]):
#         print(i, val)
        if int(params[i]) == 0:
            ppr.append(int(s[int(val)]))
        else:
            ppr.append(int(val))
            
#     print(ppr)
    
    if code == 3:
#         in_ = input()
        in_ = inp.popleft()
        in_ = int(in_)
        s[int(pr[-1])] = in_
        return True, _off
    elif code == 4:
        print('>>>', s[int(pr[-1])])
        inp.append(s[int(pr[-1])])
        return True, _off
    elif code == 5:
        flag = ppr[0]
        if flag > 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    elif code == 6:
        flag = ppr[0]
        if flag == 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    
    if code == 1:
        s[int(pr[-1])] = ppr[0] + ppr[1]
        return True, _off
    elif code == 2:
        s[int(pr[-1])] = ppr[0] * ppr[1]
        return True, _off
    elif code == 7:
        s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
        return True, _off
    elif code == 8:
        s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
        return True, _off
    elif code == 99:
        return False, 1
    else:
        print('Error')
        return False

def run_prog(prog, inp):
    offset = 0
    _continue = True
    while _continue:
        _continue, _offset = op(prog, offset, inp)
        #print('Operation at {}'.format(offset))
        offset += _offset

In [ ]:
with open('p6_input.txt') as f:
    orbits = f.read().split()

In [ ]:
orbits = [
'COM)B',
'B)C',
'C)D',
'D)E',
'E)F',
'B)G',
'G)H',
'D)I',
'E)J',
'J)K',
'K)L',
'K)YOU',
'I)SAN'
]

In [ ]:
from collections import defaultdict

fw = defaultdict(list)
bw = dict()

for orb in orbits:
    a, b = tuple(orb.split(')'))
    fw[a].append(b)
    bw[b] = a

In [ ]:
from collections import deque

def orbcounter(p, trace=False):
    cnt = 0
    tr = deque()
    tr.appendleft(p)
    while p != 'COM':
        cnt += 1
        p = bw[p]
        tr.appendleft(p)
    if trace:
        return cnt, tr
    return cnt

cnt = 0

for sp in bw.keys():
    p = sp
    cnt += orbcounter(p)
    
        
print(cnt)

In [ ]:
cnt1, tr1 = orbcounter('YOU', True)
cnt2, tr2 = orbcounter('SAN', True)

In [ ]:
bp = ''
for i in range(min(len(tr1), len(tr2))):
    if tr1[i] == tr2[i]:
        bp = tr1[i]
    else:
        break
    

In [ ]:
transfer = cnt1 + cnt2 - 2*(orbcounter(bp) + 1)
print(transfer)

## P5

In [ ]:
def opc(s):
    s_ = '{:05d}'.format(int(s))
    code = int(s_[-2:])
    params = s_[0:3]
    return code, params[::-1]

def op(s, offset):
    code, params = opc(s[offset])
    print(s[offset], code, offset, params)
    _off = 4
    if code == 3 or code == 4:
        _off = 2
    if code == 5 or code == 6:
        _off = 3
    
    if code == 99:
        return False, 1
    pr = s[offset:offset+_off].copy()
    
    ppr = []
    for i, val in enumerate(pr[1:]):
#         print(i, val)
        if int(params[i]) == 0:
            ppr.append(int(s[int(val)]))
        else:
            ppr.append(int(val))
            
#     print(ppr)
    
    if code == 3:
        in_ = input()
        in_ = int(in_)
        s[int(pr[-1])] = in_
        return True, _off
    elif code == 4:
        print('>>>', s[int(pr[-1])])
        return True, _off
    elif code == 5:
        flag = ppr[0]
        if flag > 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    elif code == 6:
        flag = ppr[0]
        if flag == 0:
            new_addr = ppr[1]
            return True, new_addr - offset
        else:
            return True, _off
    
    if code == 1:
        s[int(pr[-1])] = ppr[0] + ppr[1]
        return True, _off
    elif code == 2:
        s[int(pr[-1])] = ppr[0] * ppr[1]
        return True, _off
    elif code == 7:
        s[int(pr[-1])] = 1 if ppr[0] < ppr[1] else 0
        return True, _off
    elif code == 8:
        s[int(pr[-1])] = 1 if ppr[0] == ppr[1] else 0
        return True, _off
    elif code == 99:
        return False, 1
    else:
        print('Error')
        return False

def run_prog(prog):
    offset = 0
    _continue = True
    while _continue:
        _continue, _offset = op(prog, offset)
        #print('Operation at {}'.format(offset))
        offset += _offset

In [ ]:
with open('p5_input.txt') as f:
    prog = f.read().strip().split(',')

In [ ]:
run_prog(prog)

In [ ]:
prog[0:10]

In [ ]:
test = [1002,4,3,4,33]
run(test)

In [ ]:
test

In [ ]:
run_prog(prog1)

## P4

In [ ]:
low = 264360
up = 746325
cnt = 0
for x1 in range(8):
    for x2 in range(x1, 10):
        for x3 in range(x2, 10):
            for x4 in range(x3, 10):
                for x5 in range(x4, 10):
                    for x6 in range(x5, 10):
                        if not (x1 == x2 or x2 == x3 or x3 == x4 or x4 == x5 or x5 == x6):
                            continue
                        if not check_nb([x1, x2, x3, x4, x5, x6]):
                            continue
                        nb = x1 * 100000 + x2*10000 + x3*1000 + x4 *100 + x5 *10 + x6
                        if nb < low:
                            continue
                        if nb > up:
                            break
                        cnt += 1
print(cnt)

In [ ]:
from collections import Counter

def check_nb(l):
    c = Counter()
    last = l[0]
    c[last] += 1
    for i in range(1,6):
        c[l[i]] += 1
        last = l[i]
     
    return any(cnt == 2 for cnt in c.values())
check_nb([1,2,3,4,5,5])

In [ ]:
_.values()

## P3

In [ ]:
with open('p3_input.txt') as f:
    wires = f.read().split()
    wires = [w.split(',') for w in wires]
wires

In [ ]:
def create_path(w):
    path = [(0, 0)]
    x, y = (0,0)
    for d in w:
        o = d[0]
        amount = int(d[1:])
        for i in range(amount):
            if o=='R':
                x += 1
            elif o=='L':
                x -= 1
            elif o=='U':
                y -= 1
            elif o=='D':
                y += 1
            path.append((x, y))
    return path
p1 = create_path(wires[0])
p2 = create_path(wires[1])

In [ ]:
crossing_points = set(p1) & set(p2)

In [ ]:
crossing_points.remove((0, 0))

In [ ]:
min(abs(cp[0])+abs(cp[1]) for cp in crossing_points)

In [ ]:
lengths = []
for cp in crossing_points:
    l = 0
    for p in p1:
        if p == cp:
            break
        l+=1
    for p in p2:
        if p == cp:
            break
        l+=1
    lengths.append(l)
min(lengths)

## P1

In [ ]:
with open('p1_input.txt') as f:
    lines = f.read().split()
    
def fuel_need(val):
    return max(int(val / 3) - 2, 0)

def fuel_comp(val):
    tot = 0
    fn = fuel_need(val)
    while fn > 0:
        tot += fn
        fn = fuel_need(fn)
    return tot

In [ ]:
need = sum(fuel_need(int(val)) for val in lines)
print(need)

In [ ]:
need = sum(fuel_comp(int(val)) for val in lines)
print(need)

## P2

In [ ]:
def get_prog():
    return [1,0,0,3,1,1,2,3,1,3,4,3,1,5,0,3,2,1,13,19,2,9,19,23,1,23,6,27,1,13,27,31,1,31,10,35,1,9,35,39,1,39,9,43,2,6,43,47,1,47,5,51,2,10,51,55,1,6,55,59,2,13,59,63,2,13,63,67,1,6,67,71,1,71,5,75,2,75,6,79,1,5,79,83,1,83,6,87,2,10,87,91,1,9,91,95,1,6,95,99,1,99,6,103,2,103,9,107,2,107,10,111,1,5,111,115,1,115,6,119,2,6,119,123,1,10,123,127,1,127,5,131,1,131,2,135,1,135,5,0,99,2,0,14,0]

In [ ]:
prog[1] = 12
prog[2] = 2

In [ ]:
def op(s, offset):
    if s[offset] == 99:
        return False
    pr = s[offset:offset+4].copy()
    opc = pr[0]
    a = s[pr[1]]
    b = s[pr[2]]
    if opc==1:
        s[pr[3]] = a + b
        return True
    elif opc==2:
        s[pr[3]] = a*b
        return True
    elif opc==99:
        return False
    else:
        print('Error')
        return False

def run(prog):
    offset = 0
    while op(prog, offset):
        #print('Operation at {}'.format(offset))
        offset += 4

In [ ]:
prog

In [ ]:
test1 = [1,0,0,0,99]

In [ ]:
run(test1)

In [ ]:
run(prog)

In [ ]:
prog

In [ ]:
for a in range(100):
    for b in range(100):
        p = get_prog()
        p[1] = a
        p[2] = b
        run(p)
        if p[0] == 19690720:
            print(a, b)
            break